In [23]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from wordcloud import WordCloud, STOPWORDS
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.tokenize.toktok import ToktokTokenizer

In [24]:
data= pd.read_csv('C:/Users/hp/OneDrive/Desktop/archive (2)/Fitabase Data 4.12.16-5.12.16/IMDB Movie Reviews DataSet/IMDB Dataset.csv')
print(data.head(10))

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive


In [25]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

There are no null values in the dataset, this is expected as the data set is a popular one, which I downloaded from kaggle.

## Exploratory Data Analysis

In [26]:
data.shape

(50000, 2)

There are fifty thousand rows in the data set, with two columns

In [27]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [28]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

So, the dataset is equally divided, in terms of negative and positive reviews

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
tokenizer=ToktokTokenizer
stopwords= nltk.corpus.stopwords.words('english')

In [31]:
def remove_noise(text):
    soup= BeautifulSoup(text, 'html.parser')
    text=soup.get_text()
    text=re.sub("\[[^]]*\]",'',text)
    return text


In [32]:
data['review']= data['review'].apply(remove_noise)

C:\Users\hp\AppData\Local\Temp\ipykernel_4540\2111781767.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup= BeautifulSoup(text, 'html.parser')


In [33]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [34]:
def stemm(text):
    port= nltk.porter.PorterStemmer()
    text=' '.join([port.stem(word) for word in text.split()])
    return text

In [35]:
data['review']= data['review'].apply(stemm)

In [36]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


In [37]:
def remove_stopwords(text, is_lower_case= False):
    tokenizer=ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens=[i.strip() for i in tokens]
    if is_lower_case:
        filtokens=[i for i in tokens not in stopwords]
    else:
        filtokens=[i for i in tokens if i.lower not in stopwords]
        filtered_texts= ' '.join(filtokens)
        return filtered_texts

In [38]:
data['review']= data['review'].apply(remove_stopwords)

In [39]:
data.head(10)

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there ' a famili where a littl boy ( jak...,negative
4,"petter mattei ' "" love in the time of money "" ...",positive
5,"probabl my all-tim favorit movie , a stori of ...",positive
6,i sure would like to see a resurrect of a up d...,positive
7,"thi show wa an amazing , fresh &amp; innov ide...",negative
8,encourag by the posit comment about thi film o...,negative
9,if you like origin gut wrench laughter you wil...,positive


Split the data into train and test sets

In [40]:
train_reviews= data.review[:30000]
test_reviews = data.review[30000:]

In [41]:
tv= TfidfVectorizer(min_df=0.0, max_df=1,use_idf=True, ngram_range= (1,3))
tv_train= tv.fit_transform(train_reviews)
tv_test= tv.transform(test_reviews)

In [42]:
label= LabelBinarizer()
sentiments= label.fit_transform(data['sentiment'])

In [43]:
train_sentiments= data.sentiment[:30000]
test_sentiments= data.sentiment[30000:] 

In [44]:
from sklearn import svm
clf= svm.SVC(kernel='linear')

clf.fit(tv_train, train_sentiments)

SVC(kernel='linear')

In [45]:
y_pred= clf.predict(tv_test)
print(y_pred)

['positive' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [46]:
score= accuracy_score(test_sentiments, y_pred)
print(score)

0.75935
